In [2]:
### Load Libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
from impyute.imputation.cs import mice
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
import pickle


In [4]:
### Load data
# unpickle using gzip
usig_1 = pd.read_pickle('usig_1.gz', compression='gzip')
usig_2 = pd.read_pickle('usig_2.gz', compression='gzip')
usig_3 = pd.read_pickle('usig_3.gz', compression='gzip')
usig_4 = pd.read_pickle('usig_4.gz', compression='gzip')

In [115]:
##Concatenate data
df = pd.concat([usig_1, usig_2, usig_3, usig_4], axis=0)
df.head(5)

,AsOfDate,ISIN,Ticker,IssueDate,Coupon,YrsToMat,IssuerName,UltimateParentTicker,IndexRating,IndexRating_Num,...,IS_REG_S,Momentum: 6-Month Equity Return (CORP),Momentum Rev.: 6-Month OAS Percentage Change (CORP),Quality: Interest Coverage Ratio 2-Month Lagged (CORP),Low Volatility: OAS 12-Month Volatility (CORP),Illiquidity: Bond Amount Issuer (CORP),Value (CORP),Low Volatility: OAS 24-Month Volatility (CORP),Size: Equity Market Value (CORP),IG Composite V2
0,2011-01-31,US00077TAA25,RBS,2011-01-31,7.75,12.0,ABN-AMRO BANK NV,NaN,A2,700.0,...,False,NaN,-0.047461,NaN,-0.085100,-51252.480017,170.979567,-0.160395,NaN,NaN
1,2011-01-31,US00080QAB14,RBS,2011-01-31,4.65,7.0,ABN-AMRO BANK NV - GLOBAL,NaN,A2,700.0,...,False,NaN,0.065322,NaN,-0.085521,-51252.480017,167.459604,-0.193118,NaN,NaN
2,2011-01-31,US001055AC65,AFL,2011-01-31,8.50,8.0,AFLAC INC,NaN,A3,690.0,...,False,0.170563,-0.174160,NaN,-0.140696,-27000.000000,19.232855,NaN,-27135.210938,NaN
3,2011-01-31,US001055AD49,AFL,2011-01-31,6.90,28.0,AFLAC INC,NaN,A3,690.0,...,False,0.170563,-0.244913,NaN,-0.143054,-27000.000000,-2.784821,NaN,-27135.210938,NaN
4,2011-01-31,US001055AE22,AFL,2011-01-31,3.45,4.0,AFLAC INC,NaN,A3,690.0,...,False,NaN,NaN,NaN,NaN,-27000.000000,-48.594967,NaN,-27135.210938,NaN


In [8]:
### Convert Catergorical Ordinal Variables to Numeric

conv_dict_snp={'A':6, 'A+':5, 'A-':7, 'AA':3, 'AA+':2, 'AA-':4, 'AAA':1, 'B':15, 'B+':14, 'B-':16, 'BB':12,
       'BB+':11, 'BB-':13, 'BBB':9, 'BBB+':8, 'BBB-':10, 'NR':17}
df['SNP_Num'] = df['RatingSNP'].apply(conv_dict_snp.get)
#df[df['SNP_Num']==5].iloc[:,11:23]
conv_dict_moody={'A1':5, 'A2':6, 'A3':7, 'AA1':2, 'AA2':3, 'AA3':4, 'AAA':1, 'Aa1':2, 'Aa2':3, 'Aa3':4,
       'Aaa':1, 'B1':14, 'B2':15, 'B3':16, 'BA1':11, 'BA2':12, 'BA3':13, 'BAA1':8, 'BAA2':9, 'BAA3':10,
       'Ba1':11, 'Ba2':12, 'Ba3':13, 'Baa1':8, 'Baa2':9, 'Baa3':10, 'CAA1':17, 'NR':18}
df['Moody_Num'] = df['RatingMoody'].apply(conv_dict_moody.get)
# df[df['Moody_Num']==5].iloc[:,11:23]

# df['Barclays_2_Num'] = df['Barclays_Lv_2'].apply(barclays_1_dict.get)
# df['Barclays_3_Num'] = df['Barclays_Lv_3'].apply(barclays_2_dict.get)
# df['Barclays_4_Num'] = df['Barclays_Lv_4'].apply(barclays_3_dict.get)


In [10]:
def iterativeimputer(df):
    linearreg = LinearRegression()
    imp = IterativeImputer(estimator = linearreg, missing_values = np.nan, max_iter = 10, verbose = 2, imputation_order = 'roman', random_state= 0)
    imputed_data = imp.fit_transform(df)
    return pd.DataFrame(imputed_data,columns=df.columns)

def knnimpute(df):
    knn_imputer = KNNImputer(n_neighbors=5,weights='uniform')
    array_imputed = knn_imputer.fit_transform(df)
    df_imputed = pd.DataFrame(array_imputed, index = df.index, columns = df.columns).reset_index()
    return df_imputed

In [11]:
def null_stats(df):
    null_stats = pd.DataFrame(data=None,index=df.columns)
    for i in df.columns:
        null_stats.loc[i,'NA'] = df[i].isna().sum()
        null_stats.sort_values(by='NA',ascending=False,inplace=True)
    return null_stats
#null_stats(df_dtd_imputed)

In [116]:
df_date = df.set_index('AsOfDate')
df_date.index = df_date.index.astype('datetime64[ns]')

In [122]:
null_stats(df).sort_values(by='NA',ascending=False)

,NA
BidAsk2,593499.0
FactSet_EBITDA,371934.0
RatingFitch,294932.0
FactSet_INTEREST_EXPENSE,289690.0
FactSet_AMOUNT_OUT,286827.0
Low Volatility: OAS 24-Month Volatility (CORP),280053.0
UltimateParentTicker,279762.0
FactSet_TOTAL_DEBT,255836.0
FactSet_TOTAL_ASSETS,255214.0
Quality: Interest Coverage Ratio 2-Month Lagged (CORP),251950.0


In [124]:
### Take only the data after 2020-01-01
df_2020 = df_date[df_date.index > '2020-01-01']
null_stats(df_2020).sort_values(by='NA',ascending=False)

,NA
FactSet_AMOUNT_OUT,224444.0
FactSet_EBITDA,211948.0
FactSet_INTEREST_EXPENSE,209131.0
FactSet_TOTAL_ASSETS,208978.0
FactSet_TOTAL_DEBT,208978.0
Value (CORP),86709.0
Low Volatility: OAS 24-Month Volatility (CORP),80558.0
DTD,80402.0
Momentum: 6-Month Equity Return (CORP),78738.0
Quality: Interest Coverage Ratio 2-Month Lagged (CORP),75006.0


In [117]:
df_date.columns

Index(['ISIN', 'Ticker', 'IssueDate', 'Coupon', 'YrsToMat', 'IssuerName',
       'UltimateParentTicker', 'IndexRating', 'IndexRating_Num', 'RatingFitch',
       'RatingMoody', 'RatingSNP', 'Barclays_Lv_1', 'Barclays_Lv_2',
       'Barclays_Lv_3', 'Barclays_Lv_4', 'PriceEnd', 'OASEnd', 'OADEnd',
       'TMVEnd', 'ReturnExcess', 'RORTotal', 'BidAsk', 'FactSet_AMOUNT_OUT',
       'FactSet_EQUITY_PRICE', 'FactSet_EQUITY_MKT_VAL_MM',
       'FactSet_TOTAL_ASSETS', 'FactSet_TOTAL_DEBT', 'FactSet_EBITDA',
       'FactSet_INTEREST_EXPENSE', 'Cal_INTEREST_COVERAGE_RATIO', 'Cal_DTS',
       'Cal_LEVERAGE_RATIO', 'Cal_AMOUNT_OUT_ISSR', 'PctVol', 'CountryOfRisk',
       'CountryOfIssue', 'BidAsk2', 'DTD', 'COLLATERAL_TYPE', 'STRUCTURE',
       'REGISTRATION', 'IS_144A', 'IS_PRIVATE_PLACEMENT', 'IS_REG_S',
       'Momentum: 6-Month Equity Return (CORP)',
       'Momentum Rev.: 6-Month OAS Percentage Change (CORP)',
       'Quality: Interest Coverage Ratio 2-Month Lagged (CORP)',
       'Low Volatil

In [118]:
def cross_section_performance(df, date, imputer = iterativeimputer, feature=['DTD'],num=5):
    df_date = df.dropna(subset=[feature[0]])
    date_count = df_date.reset_index().groupby('AsOfDate')['AsOfDate'].count()
    previous_date = date_count[date_count.index.get_loc(date) - 1:date_count.index.get_loc(date)].index
    date = previous_date.format("yyyy-mm-dd")[1]
    df_date_day = df_date.loc[date]
    df_date_day_copy = df_date_day.copy()
    indexes = []
    np.random.seed(123)
    for i in range(len(df_date_day.index)):
        r = np.random.randint(1,5)
        if r==4:
            df_date_day_copy.iloc[i,df_date_day.columns.get_loc(feature[0])] = np.nan
            indexes.append(i)
    
    s = [\
        'Coupon',\
        'YrsToMat',\
        'IndexRating_Num',\
        'PriceEnd',\
        'OASEnd',\
        'OADEnd',\
        'TMVEnd',\
        'ReturnExcess',\
        'RORTotal',\
        'FactSet_AMOUNT_OUT',\
        'FactSet_EQUITY_PRICE',\
        'FactSet_EQUITY_MKT_VAL_MM',\
        'FactSet_TOTAL_ASSETS',\
        'FactSet_TOTAL_DEBT',\
        'FactSet_EBITDA',\
        'FactSet_INTEREST_EXPENSE',\
        'Cal_INTEREST_COVERAGE_RATIO',\
        'Cal_DTS',\
        'Cal_LEVERAGE_RATIO',\
        'Cal_AMOUNT_OUT_ISSR',\
        'PctVol',\
        'DTD',\
        'BidAsk2',\
        'Value (CORP)',\
        ]
    
    #s = ['OASEnd', 'OADEnd', 'FactSet_TOTAL_DEBT','IG Composite V2','Cal_AMOUNT_OUT_ISSR', 'IndexRating_Num','DTD']

    df_date_day_copy_numerical = df_date_day_copy[s]
    temp_correlation = pd.DataFrame(data=None, index = df_date_day_copy_numerical.columns)
    for i in df_date_day_copy_numerical.columns:
        #print(i,df_date_day_copy_numerical['DTD'].corr(df_date_day_copy_numerical[i]))
        temp_correlation.loc[i,"Correlation"] = abs(df_date_day_copy_numerical[feature[0]].corr(df_date_day_copy_numerical[i]))
    factors = list(temp_correlation.sort_values(by="Correlation",ascending=False).index[1:num+1])
    factors += feature
    df_date_day_copy_numerical_select = df_date_day_copy_numerical[factors]
    df_date_day_copy_numerical_select_imputed = imputer(df_date_day_copy_numerical_select)

    for i in range(len(df_date_day_copy_numerical_select_imputed)):
    
        if df_date_day_copy_numerical_select_imputed.iloc[i,num] < 0:
            df_date_day_copy_numerical_select_imputed.iloc[i,num] = 0
    
    stat_og = df_date_day.describe().loc[:,feature[0]]
    stat_imputed = df_date_day_copy_numerical_select_imputed.describe().loc[:,feature[0]]
    #print(stat_og)
    #print("\n")
    #print(stat_imputed)
    y =  df_date_day_copy_numerical_select_imputed.iloc[indexes,df_date_day_copy_numerical_select_imputed.columns.get_loc(feature[0])]
    X = df_date_day.iloc[indexes,df_date_day.columns.get_loc(feature[0])]
    X = X.reset_index().drop(columns='AsOfDate')
    y = y.reset_index().drop(columns = 'index')
    reg = sm.OLS(y,X).fit()
    return factors, stat_og, stat_imputed, reg
    
    
    

## DTD

In [ ]:
## regression r-squared for multiple factors from 3 to 10
a = 'DTD'
r2_df = pd.DataFrame(data=None, index = list(range(3,15)))
for i in range(3,15):
    _, _, _, reg1 = cross_section_performance(df_date, '2013-05-31', iterativeimputer, [a], num=i)
    _, _, _, reg2 = cross_section_performance(df_date, '2013-05-31', knnimpute, [a],num=i)
    r2_df.loc[i,'r2_MICE'] = reg1.rsquared
    r2_df.loc[i,'r2_KNN'] = reg2.rsquared
    

In [111]:
r2_df

,r2_MICE,r2_KNN
3,0.732172,0.716135
4,0.732309,0.719001
5,0.732809,0.722823
6,0.732471,0.726608
7,0.733961,0.794083
8,0.738910,0.798583
9,0.739048,0.805716
10,0.737262,0.807715
11,0.736713,0.799399
12,0.736658,0.795211


## Value Factor

In [ ]:
## regression r-squared for multiple factors from 3 to 10
a = 'Value (CORP)'
r2_df = pd.DataFrame(data=None, index = list(range(3,15)))
for i in range(3,15):
    _, _, _, reg1 = cross_section_performance(df_date, '2013-05-31', iterativeimputer, [a], num=i)
    _, _, _, reg2 = cross_section_performance(df_date, '2013-05-31', knnimpute, [a],num=i)
    r2_df.loc[i,'r2_MICE'] = reg1.rsquared
    r2_df.loc[i,'r2_KNN'] = reg2.rsquared

In [130]:
r2_df

,r2_MICE,r2_KNN
3,0.254367,0.251439
4,0.251847,0.261870
5,0.252264,0.249880
6,0.250615,0.290269
7,0.251948,0.287636
8,0.254982,0.263103
9,0.256686,0.267054
10,0.266991,0.306237
11,0.267352,0.001188
12,0.266959,0.001081


## 'BidAsk2'

In [ ]:
## regression r-squared for multiple factors from 3 to 10
a = 'BidAsk2'
r2_df = pd.DataFrame(data=None, index = list(range(3,15)))
for i in range(3,15):
    _, _, _, reg1 = cross_section_performance(df_date, '2021-05-31', iterativeimputer, [a], num=i)
    _, _, _, reg2 = cross_section_performance(df_date, '2021-05-31', knnimpute, [a],num=i)
    r2_df.loc[i,'r2_MICE'] = reg1.rsquared
    r2_df.loc[i,'r2_KNN'] = reg2.rsquared

In [132]:
r2_df

,r2_MICE,r2_KNN
3,0.758560,0.736837
4,0.766318,0.736807
5,0.768751,0.751463
6,0.785566,0.755918
7,0.785620,0.755905
8,0.801612,0.676114
9,0.802739,0.676114
10,0.804583,0.531772
11,0.804682,0.531772
12,0.804513,0.531772


# DTD NEW

In [183]:
def iterativeimputer(df):
    linearreg = LinearRegression()
    imp = IterativeImputer(estimator = linearreg, missing_values = np.nan, max_iter = 10, verbose = 2, imputation_order = 'roman', random_state= 0)
    imputed_data = imp.fit_transform(df)
    return pd.DataFrame(imputed_data,columns=df.columns)

In [198]:
feature = ['DTD']
df_date_1 = df_date.dropna(subset=[feature[0]])
df_date_1 = df_date_1.reset_index()
df_filter = df_date_1[df_date_1['AsOfDate'] == '2013-05-31']
df_filter_1 = df_filter.copy()

#Count unique tickers
count = df_filter['Ticker'].nunique()

tickers_to_drop = []
unique_tickers = df_filter['Ticker'].unique()
np.random.seed(123)
for i in range(1,count):
    r = np.random.randint(1,5)
    if r == 4:
        tickers_to_drop.append(unique_tickers[i])

# if ticker_to_drop then change ['DTD'] to nan
df_filter.loc[df_filter['Ticker'].isin(tickers_to_drop),['DTD']] = np.nan

s = ['DTD','OASEnd', 'OADEnd', 'FactSet_TOTAL_DEBT','Cal_AMOUNT_OUT_ISSR', 'IndexRating_Num','IG Composite V2']

df_filter_numerical = df_filter[s]
df_filter_1_numerical = df_filter_1[s]

linearreg = LinearRegression()
imp = IterativeImputer(estimator = linearreg, missing_values = np.nan, max_iter = 10, verbose = 2, imputation_order = 'roman', random_state= 0)
imputed_data = imp.fit_transform(df_filter_numerical)
df_filter_numerical_imputed = pd.DataFrame(imputed_data,columns=df_filter_numerical.columns[:-1])
df_filter_numerical_imputed

knn_imputer = KNNImputer(n_neighbors=5,weights='uniform')
array_imputed = knn_imputer.fit_transform(df_filter_numerical)
df_filter_numerical_imputed_knn = pd.DataFrame(array_imputed, index = df_filter_numerical.index, columns = df_filter_numerical.columns[:-1]).reset_index()
df_filter_numerical_imputed_knn

/Users/amankrishna/miniforge3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


[IterativeImputer] Completing matrix with shape (4348, 6)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.01
[IterativeImputer] Change: 1337547.3830939494, scaled tolerance: 43798.832 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.01
[IterativeImputer] Change: 87090.89440412144, scaled tolerance: 43798.832 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.02
[IterativeImputer] Change: 6029.362259409038, scaled tolerance: 43798.832 
[IterativeImputer] Early stopping criterion reached.


,index,DTD,OASEnd,OADEnd,FactSet_TOTAL_DEBT,Cal_AMOUNT_OUT_ISSR,IndexRating_Num
0,103815,10.496200,83.525902,3.831038,9218.490234,2500.000000,700.0
1,103816,10.496200,98.420101,8.075648,9218.490234,2500.000000,700.0
2,103817,10.496200,115.388596,16.974468,9218.490234,2500.000000,700.0
3,103818,1.688280,325.028610,4.445928,140356.000000,63251.542925,660.0
4,103819,12.329970,129.662001,4.977798,4352.000000,61000.000000,690.0
...,...,...,...,...,...,...,...
4343,108158,16.306641,133.009696,7.539604,1820.900024,1550.000000,680.0
4344,108159,16.306641,61.304098,1.486012,1820.900024,1550.000000,680.0
4345,108160,4.115908,136.049497,1.257976,42897.879187,14711.287997,660.0
4346,108161,6.278882,170.672297,3.543032,14353.328406,6377.543597,660.0


In [203]:
df_filter_1_numerical.shape

(4348, 7)

In [204]:
df_filter_numerical_imputed_knn.shape

(4348, 7)

In [214]:
# calulare r-squared of df_filter_numerical_imputed['DTD'] vs df_date_1_numerical['DTD']
y = pd.DataFrame(df_filter_numerical_imputed_knn['DTD'])
X = pd.DataFrame(df_filter_1_numerical['DTD'])
X = X.reset_index().drop(columns='index')
reg = sm.OLS(y,X).fit()
reg.rsquared

0.9237870324453276

In [211]:
y = pd.DataFrame(df_filter_numerical_imputed['DTD'])
X = pd.DataFrame(df_filter_1_numerical['DTD'])
X = X.reset_index().drop(columns='index')
reg = sm.OLS(y,X).fit()
reg.rsquared

0.9328108149134905